# Simple Compound Device

## Configuration

This code would normally go in a script automatically run at startup. The user would not have to worry about this.

In [ ]:
%matplotlib notebook
%run startup.py

## Data Acquisition

### Signals

This is a simple Signal. We can read it and set it.

In [ ]:
from ophyd import Signal

x = Signal(value=0, name='x')

In [ ]:
x.read()

In [ ]:
x.set(3)

In [ ]:
x.read()

The RunEngine calls these methods on ``x`` for us when we scan it:

In [ ]:
from ophyd.sim import det

RE(scan([], x, -1, 1, 3))

The ``Signal`` class is a "soft" signal not communicating with physical hardware. At beamlines, we more often use ``EpicsSignal`` which takes a PV string as its argument.

```python
x = EpicsSignal('PV:...', name ='x')
```

### Device

A Device is composed of Signals (or other Devices) that are bundled to create logical groups that map to our notion of a physical piece of hardware -- for example, an XY stage.

In [ ]:
from ophyd import Device, Component as Cpt

class XYStage(Device):
    x = Cpt(Signal, value=0)
    y = Cpt(Signal, value=0)


stage = XYStage(name='stage')

The readings from x and y are collated into one dictionary. The names of the components ('x' and 'y') are joined with the name of the overall device ('stage').

In [ ]:
stage.read()

Each component signal can still be read or set individually.

In [ ]:
stage.x.set(2)
stage.x.read()

In [ ]:
stage.read()

### Built-in Devices

Ophyd includes some built-in Devices for EPICS. For example, ``EpicsMotor`` looks something like this:

```python
class EpicsMotor(Device):
    # position
    user_readback = Cpt(EpicsSignalRO, '.RBV')
    user_setpoint = Cpt(EpicsSignal, '.VAL', limits=True)

    # calibration dial <-> user
    user_offset = Cpt(EpicsSignal, '.OFF')
    user_offset_dir = Cpt(EpicsSignal, '.DIR')
    offset_freeze_switch = Cpt(EpicsSignal, '.FOFF')
    set_use_switch = Cpt(EpicsSignal, '.SET')

    # configuration
    velocity = Cpt(EpicsSignal, '.VELO')
    acceleration = Cpt(EpicsSignal, '.ACCL')
    motor_egu = Cpt(EpicsSignal, '.EGU')

    # motor status
    motor_is_moving = Cpt(EpicsSignalRO, '.MOVN')
    motor_done_move = Cpt(EpicsSignalRO, '.DMOV')
    high_limit_switch = Cpt(EpicsSignal, '.HLS')
    low_limit_switch = Cpt(EpicsSignal, '.LLS')
    direction_of_travel = Cpt(EpicsSignal, '.TDIR')

    # commands
    motor_stop = Cpt(EpicsSignal, '.STOP')
    home_forward = Cpt(EpicsSignal, '.HOMF')
    home_reverse = Cpt(EpicsSignal, '.HOMR')

    ...
```

Connecting to a specific motor looks like:

```python
motor = EpicsMotor('PV:...')
```

Likewise, connecting to an area detector is simple:

```python
ccd = Prosilica('PV:...')  # an area detector
```

### Using the custom XY Stage Device in a raster scan

In [ ]:
from bluesky.plans import outer_product_scan, subs_wrapper

# Define a simulated detector that is coupled to our simulated XY stage.
det = Syn2DGauss('det',
                 stage.x, 'stage_x',
                 stage.y, 'stage_y',
                 center=(1, -1),
                 Imax=1000,
                 sigma=10,
                 noise='poisson')
det.exposure_time = 0.1

def raster_scan(x_start, x_stop, NX,
                y_start, y_stop, NY):
    "A variation on outer_product_scan that includes LiveGrid and LiveTable built in."
    fig, ax = plt.subplots()
    lg = LiveGrid((NY, NX), 'det',
                  xlabel='stage_x', ylabel='stage_y',
                  ax=ax,
                  extent=(x_start, x_stop,
                          y_start, y_stop))
    lt = LiveTable([det, stage])

    return (yield from subs_wrapper(
        outer_product_scan([det],
                  stage.y, y_start, y_stop, NY,
                  stage.x, x_start, x_stop, NX,
                  True),
        [lg, lt]))

In [ ]:
RE(raster_scan(-1, 1, 3, -1, 1, 3))

### Using the custom XY Stage Device for multiple 1D scans

In [ ]:
class WaterfallLivePlot(LivePlot):
    "Plot multiple lines on the same axes, offset in y."
    def __init__(self, *args, offset_scale=1, **kwargs):
        self.offset_scale = offset_scale
        self.count = -1
        super().__init__(*args, **kwargs)

    def start(self, doc):
        self.count += 1
        return super().start(doc)

    def update_caches(self, x, y):
        return super().update_caches(x, y + self.offset_scale*self.count)


def line_scans(x_start, x_stop, NX,
               y_start, y_stop, NY):
    "Perform multiple 1D scans over x, stepping through y for each scan."
    fig, ax = plt.subplots()
    lp = WaterfallLivePlot(y='det', x='stage_x',
                           offset_scale=100, ax=ax,
                           legend_keys=['y_shift'])
    lt = LiveTable([det, stage])
    uids = []
    for y in np.linspace(y_start, y_stop, NY):
        yield from mv(stage.y, y)
        uid = yield from subs_wrapper(
            scan([det, stage],
                 stage.x, x_start, x_stop, NX,
                 md={'y_shift': y}),
            [lp, lt])

    return uids

In [ ]:
RE(line_scans(1, 10, 10, 1, 5, 5))

## Exercises

1. Define a new custom Device representing slits with four degrees of free (four signals) named ``top``, ``bottom``, ``inner``, and ``outer``. Read the device.
2. Define a new custom Device whose components are Devices. Make an ``Endstation`` Device with four components: two XYStages and two slits.